<a href="https://colab.research.google.com/github/AslanDevbrat/Seq2Seq/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character-level recurrent sequence-to-sequence model

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2017/09/29<br>
**Last modified:** 2020/04/26<br>
**Description:** Character-level recurrent sequence-to-sequence model.

## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.


## Setup


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, SimpleRNNCell, GRUCell, Dense, LSTMCell
from tensorflow.keras import Input
import pandas as pd
from numpy import argmax
from math import log

## Download the data


## Configuration


In [7]:
!wget  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'
train_file_path = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
val_file_path= "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"
test_file_path  = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"

--2022-06-18 17:53:08--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.128, 142.250.97.128, 74.125.196.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   165MB/s    in 11s     

2022-06-18 17:53:19 (171 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [102]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = train_file_path


## Prepare the data


In [117]:
def processData(filename,input_chars=set(),target_chars=set()):
  input=[]
  target=[]
  with open(filename, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:100]:
      t_text,i_text, attestation = line.split("\t")
       # We use "\t" as the "start sequence" character and "\n" as "end sequence" character for the target text.
      input.append(i_text)
      target.append("\t"+t_text+"\n")
      for char in i_text:
        if char not in input_chars:
            input_chars.add(char)
      for char in t_text:
        if char not in target_chars:
            target_chars.add(char)
  target_chars.add("\t")
  target_chars.add("\n")

  input_chars = sorted(list(input_chars))
  target_chars = sorted(list(target_chars))
  num_encoder_tokens = len(input_chars)
  num_decoder_tokens = len(target_chars)
  max_encoder_seq_length = max([len(txt) for txt in input])
  max_decoder_seq_length = max([len(txt) for txt in target])
  return input,target,input_chars,target_chars,num_encoder_tokens,num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length     

In [118]:
# Vectorize the data.
input,target,input_chars,target_chars,num_encoder_tokens,num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length=processData(train_file_path)
print("Number of samples:", len(input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 100
Number of unique input tokens: 23
Number of unique output tokens: 34
Max sequence length for inputs: 18
Max sequence length for outputs: 17


## Build the model


In [119]:
# Vectorize the data.
# Vectorize the data.
validation_input,validation_target,input_chars,target_chars,num_encoder_tokens,num_decoder_tokens, validation_max_encoder_seq_length, validation_max_decoder_seq_length=processData(val_file_path,set(input_chars),set(target_chars))

print("Number of validation samples:", len(validation_input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("validation Max sequence length for inputs:", validation_max_encoder_seq_length)
print("validation Max sequence length for outputs:", validation_max_decoder_seq_length)

Number of validation samples: 100
Number of unique input tokens: 23
Number of unique output tokens: 46
validation Max sequence length for inputs: 12
validation Max sequence length for outputs: 13


In [120]:
# Vectorize the data.
test_input,test_target,test_input_chars,test_target_chars,test_num_encoder_tokens,test_num_decoder_tokens, test_max_encoder_seq_length, test_max_decoder_seq_length=processData(test_file_path)
print("Number of validation samples:", len(test_input))
print("Test Max sequence length for inputs:", test_max_encoder_seq_length)
print("Test Max sequence length for outputs:", test_max_decoder_seq_length)

Number of validation samples: 100
Test Max sequence length for inputs: 12
Test Max sequence length for outputs: 12


In [121]:
input_token = dict([(char, i) for i, char in enumerate(input_chars)])
target_token = dict([(char, i) for i, char in enumerate(target_chars)])

reverse_input_token = dict((i, char) for char, i in input_token.items())
reverse_target_token = dict((i, char) for char, i in target_token.items())


encoder_input_data = np.zeros(
    (len(input), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
validation_encoder_input_data=np.zeros(
    (len(validation_input), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
test_encoder_input_data=np.zeros(
    (len(test_input), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
validation_decoder_input_data =np.zeros(
    (len(validation_input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
validation_decoder_target_data = np.zeros(
    (len(validation_input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input, target)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token[char]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token[char]] = 1.0
# for validation data
for i, (validation_input_text, validation_target_text) in enumerate(zip(validation_input, validation_target)):
    for t, char in enumerate(validation_input_text):
        validation_encoder_input_data[i, t, input_token[char]] = 1.0
    for t, char in enumerate(validation_target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        validation_decoder_input_data[i, t, target_token[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            validation_decoder_target_data[i, t - 1, target_token[char]] = 1.0

# for test data
for i, (test_input_text, test_target_text) in enumerate(zip(test_input, test_target)):
    for t, char in enumerate(test_input_text):
        test_encoder_input_data[i, t, input_token[char]] = 1.0

In [122]:
encoder_input_data.shape

(100, 18, 23)

In [123]:
class Seq2seq(tf.keras.Model):
  def __init__(self, num_encoder_tokens, num_decoder_tokens,embedding_dim,num_of_layers,unit_type, dropout , recurrent_dropout):
    super().__init__()
    self.encoder_inputs = Input(shape = (None,num_encoder_tokens))
    self.decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
    self.num_encoder_tokens = num_encoder_tokens
    self.embedding_dim = embedding_dim
    self.dropout = dropout
    self.recurrent_dropout = recurrent_dropout
    self.num_decoder_tokens = num_decoder_tokens
    self.num_of_encoder_layer  =num_of_layers
    self.num_of_decoder_layer =num_of_layers
    self.type_encoder_unit =unit_type 
    self.type_decoder_unit =unit_type
    self.train_step()
    self.build_model()

  def get_embedding_layer(self, num_encoder_tokens, embedding_dim):
    return Embedding(num_encoder_tokens, embedding_dim, )

  def get_cell(self, cell_type = "lstm", num_of_cell = 1, name = None):
    #print(cell_type)
    if cell_type == "lstm":
      return LSTMCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
    elif cell_type == "rnn":
      return SimpleRNNCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
    elif cell_type =="gru":
      return GRUCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
    else:
      print(f"Invalid cell type: {cell_type}")
  def get_encoder(self,latent_dim, cell_type = "lstm", num_of_layer = 1, ):
    return tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [self.get_cell(cell_type, latent_dim) for i in range(num_of_layer)]), return_state = True,)

  def get_decoder(self,latent_dim ,cell_type = "lstm", num_of_layer = 1, name = None ):
    return tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [self.get_cell(cell_type, latent_dim,) for i in range(num_of_layer)]), return_sequences=True, return_state=True)

  def get_dense_layer(self, num_decoder_token, activation = "softmax"):
    return Dense(num_decoder_tokens, activation= activation)

  def train_step(self):
    # self.embedding_layer = self.get_embedding_layer( self.num_encoder_tokens, self.embedding_dim)
    # self.embedding_results = self.embedding_layer()

    self.encoder = self.get_encoder( self.embedding_dim,self.type_encoder_unit, self.num_of_encoder_layer , )
    encoder_results = self.encoder(self.encoder_inputs)

    self.encoder_outputs, self.encoder_states = encoder_results[0], encoder_results[1:]

    # self.embedding_layer2 = self.get_embedding_layer( self.num_decoder_tokens, self.embedding_dim)
    # self.embedding_results2 = self.embedding_layer2(self.decoder_inputs,)

    self.decoder = self.get_decoder( self.embedding_dim, self.type_decoder_unit, self.num_of_decoder_layer,)
    self.decoder_results = self.decoder(self.decoder_inputs, initial_state=self.encoder_states)

    self.decoder_output = self.decoder_results[0]
    self.decoder_dense = self.get_dense_layer(self.num_decoder_tokens)
    self.dense_output = self.decoder_dense(self.decoder_output)

  def build_model(self):
    
    self.model = keras.Model([self.encoder_inputs, self.decoder_inputs], self.dense_output, name = "Seq2Seq_model")
    return self.model



In [124]:

seq2seq = Seq2seq(num_encoder_tokens,num_decoder_tokens, 64,4,"lstm", 0.0, 0.0).build_model()
seq2seq.summary()

Model: "Seq2Seq_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, 23)]   0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None, 46)]   0           []                               
                                                                                                  
 rnn_4 (RNN)                    [(None, 64),         121600      ['input_5[0][0]']                
                                 [(None, 64),                                                     
                                 (None, 64)],                                                     
                                 [(None, 64),                                         

In [ ]:
def lstm_cell():
  return tf.keras.layers.LSTMCell(4)

In [ ]:
num_encoder_tokens

32

In [5]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 32)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 25)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        295936      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
encoder_inputs

<KerasTensor: shape=(None, None, 32) dtype=float32 (created by layer 'input_2')>

In [ ]:
model.summary()

In [ ]:
model.summary()

## Train the model


In [224]:
predict = seq2seq( [encoder_input_data[2:3], decoder_input_data[2:3]],
    decoder_target_data[2:3])

In [225]:
predict[0].shape

TensorShape([21, 68])

In [25]:
class BeamSearch(keras.callbacks.Callback):

  def __init__(self, beam_size):
    self.beam_size = beam_size

  def beam_search_decoder(aelf, data, k):
    sequences = [[list(), 0.0]]
    # walk over each step in sequence
    for row in data:
      all_candidates = list()
      # expand each current candidate
      for i in range(len(sequences)):
        seq, score = sequences[i]
        for j in range(len(row)):
          candidate = [seq + [j], score - log(row[j])]
          all_candidates.append(candidate)
      # order all candidates by score
      ordered = sorted(all_candidates, key=lambda tup:tup[1])
      # select k best
      sequences = ordered[:k]
    return sequences
  
  def on_epoch_end(self, epoch, logs = None):
    prediction = self.model.predict([validation_encoder_input_data , validation_decoder_input_data])
    print(prediction.shape)
    for i, pred in enumerate(prediction):
      beam_search_prediction = self.beam_search_decoder(pred, self.beam_size)
      correct_prediction = 0
      for k in range(self.beam_size):
        translated_word = "\t"+"".join([reverse_target_token[x] for x in beam_search_prediction[k][0]])
        #print(translated_word)
        if translated_word == validation_target[i]:
          #print("fuck yeah")
          correct_prediction+=1
          break
    print(f"Accuracy by Beam Search {correct_prediction/len(validation_target)}")
      # print(len(beam_search_prediction))
      # print(beam_search_prediction)


In [1]:
%%capture
!pip install wandb --upgrade

In [157]:
sweep_config = {
    
    'method':'bayes',
    'metric': {
        'name':'val_accuracy',
        'goal':'maximize'
    },
    'parameters':{
    
    "num_of_layer" : {'values': [1,2,3]},
    "unit_size": {"values":[16,32,64]},
    "unit_type": {"values":["lstm","rnn","gru"]},
    "dropout": {"values": [0.0, 0.2, 0.4]},
    'recurrent_dropout':{'values':[0.0,0.3]},
    "beam_size" : {"values":[1,2,3,4]},
    "epochs":{"value":2}               
                   }
}

import pprint

pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'beam_size': {'values': [1, 2, 3, 4]},
                'dropout': {'values': [0.0, 0.2, 0.4]},
                'epochs': {'value': 2},
                'num_of_layer': {'values': [1, 2, 3]},
                'recurrent_dropout': {'values': [0.0, 0.3]},
                'unit_size': {'values': [16, 32, 64]},
                'unit_type': {'values': ['lstm', 'rnn', 'gru']}}}


In [151]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

True

In [139]:
sweep_id = wandb.sweep(sweep_config, project="seq2seq")

Create sweep with ID: 6fur9pnn
Sweep URL: https://wandb.ai/aslan/seq2seq/sweeps/6fur9pnn


In [143]:
print(history.history.keys())

NameError: ignored

In [ ]:
def train(config = None):
  with wandb.init(config=config):
    config = wandb.config
    #print(config)
    seq2seq = Seq2seq(num_encoder_tokens,num_decoder_tokens, config.unit_size, config.num_of_layer,config.unit_type , config.dropout,config.recurrent_dropout).build_model()
    seq2seq.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy",])
    seq2seq.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=config.epochs,
        validation_data =  ([validation_encoder_input_data , validation_decoder_input_data] ,validation_decoder_target_data),
        callbacks = [BeamSearch(config.beam_size), WandbCallback()],verbose = 1, 
        )


    
    
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 324vtdkz with config:
wandb: 	beam_size: 4
wandb: 	dropout: 0.4
wandb: 	epochs: 2
wandb: 	num_of_layer: 3
wandb: 	recurrent_dropout: 0
wandb: 	unit_size: 32
wandb: 	unit_type: gru


Epoch 1/2
2/2 [==============================] - ETA: 0s - loss: 1.7899 - accuracy: 0.0429(100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 11s 2s/step - loss: 1.7899 - accuracy: 0.0429 - val_loss: 1.5463 - val_accuracy: 0.0688 - _timestamp: 1655580722.0000 - _runtime: 12.0000
Epoch 2/2
2/2 [==============================] - ETA: 0s - loss: 1.7721 - accuracy: 0.0888(100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 1s 481ms/step - loss: 1.7721 - accuracy: 0.0888 - val_loss: 1.5318 - val_accuracy: 0.0829 - _timestamp: 1655580723.0000 - _runtime: 13.0000


accuracy,▁█
epoch,▁█
loss,█▁
val_accuracy,▁█
val_loss,█▁
accuracy,0.08882
best_epoch,1
best_val_loss,1.53178
epoch,1
loss,1.77208
val_accuracy,0.08294


wandb: Agent Starting Run: 2jm9p11k with config:
wandb: 	beam_size: 4
wandb: 	dropout: 0.4
wandb: 	epochs: 2
wandb: 	num_of_layer: 2
wandb: 	recurrent_dropout: 0.3
wandb: 	unit_size: 32
wandb: 	unit_type: lstm


Epoch 1/2
2/2 [==============================] - ETA: 0s - loss: 1.7909 - accuracy: 0.0218 (100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 15s 2s/step - loss: 1.7909 - accuracy: 0.0218 - val_loss: 1.5504 - val_accuracy: 0.0229 - _timestamp: 1655580747.0000 - _runtime: 17.0000
Epoch 2/2
2/2 [==============================] - ETA: 0s - loss: 1.7818 - accuracy: 0.0671(100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 1s 480ms/step - loss: 1.7818 - accuracy: 0.0671 - val_loss: 1.5429 - val_accuracy: 0.0212 - _timestamp: 1655580747.0000 - _runtime: 17.0000


...

wandb: Agent Starting Run: a4qq1jxc with config:
wandb: 	beam_size: 4
wandb: 	dropout: 0
wandb: 	epochs: 2
wandb: 	num_of_layer: 2
wandb: 	recurrent_dropout: 0.3
wandb: 	unit_size: 16
wandb: 	unit_type: gru


Epoch 1/2
2/2 [==============================] - ETA: 0s - loss: 1.7914 - accuracy: 0.0100(100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 11s 2s/step - loss: 1.7914 - accuracy: 0.0100 - val_loss: 1.5501 - val_accuracy: 0.0329 - _timestamp: 1655581798.0000 - _runtime: 17.0000
Epoch 2/2
2/2 [==============================] - ETA: 0s - loss: 1.7832 - accuracy: 0.0541(100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 1s 487ms/step - loss: 1.7832 - accuracy: 0.0541 - val_loss: 1.5457 - val_accuracy: 0.0324 - _timestamp: 1655581799.0000 - _runtime: 18.0000


accuracy,▁█
epoch,▁█
loss,█▁
val_accuracy,█▁
val_loss,█▁
accuracy,0.05412
best_epoch,1
best_val_loss,1.5457
epoch,1
loss,1.78323
val_accuracy,0.03235


wandb: Agent Starting Run: dsmcop94 with config:
wandb: 	beam_size: 4
wandb: 	dropout: 0.4
wandb: 	epochs: 2
wandb: 	num_of_layer: 1
wandb: 	recurrent_dropout: 0.3
wandb: 	unit_size: 16
wandb: 	unit_type: gru


Epoch 1/2
2/2 [==============================] - ETA: 0s - loss: 1.7997 - accuracy: 0.0094(100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 8s 2s/step - loss: 1.7997 - accuracy: 0.0094 - val_loss: 1.5588 - val_accuracy: 0.0041 - _timestamp: 1655581825.0000 - _runtime: 13.0000
Epoch 2/2
1/2 [==============>...............] - ETA: 0s - loss: 1.8599 - accuracy: 0.0285(100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 0s 435ms/step - loss: 1.7915 - accuracy: 0.0300 - val_loss: 1.5551 - val_accuracy: 0.0041 - _timestamp: 1655581826.0000 - _runtime: 14.0000


In [127]:
seq2seq.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')]
)
seq2seq.metrics_names

[]

In [158]:

histotry = seq2seq.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=2,
    validation_data =  ([validation_encoder_input_data , validation_decoder_input_data] ,validation_decoder_target_data),
    callbacks = [BeamSearch(3)]
)
# Save model
seq2seq.save("s2s")


Epoch 1/2
2/2 [==============================] - ETA: 0s - loss: 1.3460 - acc: 0.1753(100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 1s 494ms/step - loss: 1.3460 - acc: 0.1753 - val_loss: 1.3198 - val_acc: 0.1171
Epoch 2/2
2/2 [==============================] - ETA: 0s - loss: 1.3180 - acc: 0.1718(100, 17, 46)
Accuracy by Beam Search 0.0
2/2 [==============================] - 1s 429ms/step - loss: 1.3180 - acc: 0.1718 - val_loss: 1.3007 - val_acc: 0.1329


INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [161]:
for key in histotry.history.keys():
      print(key , histotry.history[key])
      #wandb.log({key : histotry.history[key]})

loss [1.3459969758987427, 1.3179850578308105]
acc [0.17529411613941193, 0.17176470160484314]
val_loss [1.3198268413543701, 1.3006969690322876]
val_acc [0.11705882102251053, 0.13294117152690887]


In [129]:
seq2seq.metrics_names

['loss', 'acc']

## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [ ]:
next(iter(train_dataset))[1]

<tf.Tensor: shape=(21,), dtype=int32, numpy=
array([ 1, 31, 11,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)>

In [ ]:
next(iter(train_dataset))[1]

<tf.Tensor: shape=(21,), dtype=int32, numpy=
array([ 1, 31, 11,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)>

In [ ]:
encoder_input_data[43].shape

(15, 32)

In [209]:
model_x.summary()

Model: "Seq2Seq_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_96 (InputLayer)          [(None, None, 28)]   0           []                               
                                                                                                  
 input_97 (InputLayer)          [(None, None, 68)]   0           []                               
                                                                                                  
 rnn_47 (RNN)                   [(None, 64),         122880      ['input_96[0][0]']               
                                 [(None, 64),                                                     
                                 (None, 64)],                                                     
                                 [(None, 64),                                         

In [208]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model_x = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
temp = model.layers[2].output
encoder_outputs, state = temp[0], temp[1:]  # lstm_1
encoder_states = state
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = state
decoder_lstm = model.layers[3]
temp = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_outputs, state_dec = temp[0], temp[1:]
decoder_states = state_dec
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
# reverse_input_char_index = dict((i, char) for char, i in num_encoder_tokens.items())
# reverse_target_char_index = dict((i, char) for char, i in num_decoder_tokens.items())
# print(reverse_input_char_index)
# print(input_token_index)

reverse_input_token = dict((i, char) for char, i in input_token.items())
reverse_target_token = dict((i, char) for char, i in target_token.items())
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        temp = decoder_model.predict([target_seq] + states_value)
        output_tokens, state = temp[0],temp[1:]

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        #print(reverse_target_char_index)
        sampled_char = reverse_target_token[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = state
    return decoded_sentence



You can now generate decoded sentences as such:


In [207]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: an
Decoded sentence: अ

-
Input sentence: ankganit
Decoded sentence: अननतत

-
Input sentence: uncle
Decoded sentence: उननल

-
Input sentence: ankur
Decoded sentence: अनाा

-
Input sentence: ankuran
Decoded sentence: अन्काा

-
Input sentence: ankurit
Decoded sentence: अन्तता

-
Input sentence: aankush
Decoded sentence: आनाा

-
Input sentence: ankush
Decoded sentence: अनाक

-
Input sentence: ang
Decoded sentence: अं

-
Input sentence: anga
Decoded sentence: अं

-
Input sentence: agandh
Decoded sentence: आआाा

-
Input sentence: angad
Decoded sentence: अंा

-
Input sentence: angane
Decoded sentence: अनना

-
Input sentence: angbhang
Decoded sentence: अंिना

-
Input sentence: angarakshak
Decoded sentence: अं्कााााा

-
Input sentence: angrakshak
Decoded sentence: अं््काााा

-
Input sentence: angara
Decoded sentence: अंाा

-
Input sentence: angaare
Decoded sentence: अनाा

-
Input sentence: angare
Decoded sentence: अंा

-
Input sentence: angi
Decoded sentence: अं

